# Microsoft GraphRAG：使用知识图谱增强检索增强生成

 
## 概述

 
Microsoft GraphRAG 是一种先进的检索增强生成 (RAG) 系统，它集成了知识图谱以提高大型语言模型 (LLM) 的性能。GraphRAG 由 Microsoft Research 开发，通过使用 LLM 生成的知识图谱来增强文档分析和提高响应质量，从而解决了传统 RAG 方法的局限性。

## 动机

 
传统的 RAG 系统通常难以处理需要综合来自不同来源信息的复杂查询。GraphRAG 旨在：
连接跨数据集的相关信息。
增强对语义概念的理解。
提高全局意义构建任务的性能。

## 关键组件

知识图谱生成：构建以实体为节点、关系为边的图。
社区检测：识别图内相关实体的集群。
摘要生成：为每个社区生成摘要，为 LLM 提供上下文。
查询处理：利用这些摘要增强 LLM 回答复杂问题的能力。
## 方法详情

索引阶段

 
文本分块：将源文本拆分为可管理的块。
元素提取：使用 LLM 识别实体和关系。
图构建：从提取的元素构建图。
社区检测：应用 Leiden 等算法查找社区。
社区摘要：为每个社区创建摘要。

查询阶段

 
本地答案生成：使用社区摘要生成初步答案。
全局答案综合：结合本地答案形成全面的响应。


## GraphRAG 的优势
GraphRAG 是一个强大的工具，它解决了基线 RAG 模型的一些关键限制。与标准 RAG 模型不同，GraphRAG 擅长识别不同信息片段之间的联系并从中得出见解。这使其成为需要从大型数据集合或难以摘要的文档中提取见解的用户的理想选择。通过利用其先进的基于图的架构，GraphRAG 能够提供对复杂语义概念的整体理解，使其成为任何需要快速准确查找信息的人的宝贵工具。无论您是研究人员、分析师，还是只是需要保持信息灵通的人，GraphRAG 都可以帮助您连接点滴，发现新的见解。

## 结论

Microsoft GraphRAG 代表了检索增强生成的重大进步，特别是对于需要对数据集有全局理解的任务。通过整合知识图谱，它提供了更高的性能，使其成为复杂信息检索和分析的理想选择。

对于有经验的基础 RAG 系统用户，GraphRAG 提供了探索更复杂解决方案的机会，尽管它可能并非适用于所有用例。
检索增强生成 (RAG) 通常通过将长文本分块，为每个块创建文本嵌入，并根据与查询的相似性搜索检索块以包含在 LLM 生成上下文中来执行。这种方法在许多场景中效果很好，并且具有引人注目的速度和成本权衡，但在需要对文本进行详细理解的场景中并不总是能很好地应对。

GraphRag ( [microsoft.github.io/graphrag](https://microsoft.github.io/graphrag/) )

<div style="text-align: center;">

<img src="../images/Microsoft_GraphRag.svg" alt="adaptive retrieval" style="width:100%; height:auto;">
</div>

要运行此笔记本，您可以使用 OpenAI API 密钥或 Azure OpenAI 密钥。
创建一个 `.env` 文件并填写您的 OpenAI 或 Azure Open AI 部署的凭据。以下代码加载这些环境变量并设置我们的 AI 客户端。


In [ ]:
AZURE_OPENAI_API_KEY=""
AZURE_OPENAI_ENDPOINT=""
GPT4O_MODEL_NAME="gpt-4o"
TEXT_EMBEDDING_3_LARGE_DEPLOYMENT_NAME=""
AZURE_OPENAI_API_VERSION="2024-06-01"

OPENAI_API_KEY=""

In [ ]:
%pip install graphrag

# 包安装和导入

下面的单元格安装运行此笔记本所需的所有必要包。


In [ ]:
# Install required packages
!pip install beautifulsoup4 openai python-dotenv pyyaml

# 包安装

下面的单元格安装运行此笔记本所需的所有必要包。如果您在一个新环境中运行此笔记本，请先执行此单元格以确保所有依赖项都已安装。

In [ ]:
# Install required packages
!pip install openai python-dotenv

In [1]:
from dotenv import load_dotenv
import os
load_dotenv()
from openai import AzureOpenAI, OpenAI

AZURE=True #Change to False to use OpenAI
if AZURE:
    AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
    AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
    GPT4O_DEPLOYMENT_NAME = os.getenv("GPT4O_MODEL_NAME")
    TEXT_EMBEDDING_3_LARGE_NAME = os.getenv("TEXT_EMBEDDING_3_LARGE_DEPLOYMENT_NAME")
    AZURE_OPENAI_API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION")
    oai = AzureOpenAI(azure_endpoint=AZURE_OPENAI_ENDPOINT, api_key=AZURE_OPENAI_API_KEY, api_version=AZURE_OPENAI_API_VERSION)
else:
    OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
    oai = OpenAI(api_key=OPENAI_API_KEY)


我们将从获取一段文本开始。维基百科上关于埃隆·马斯克的文章

In [2]:
import requests
from bs4 import BeautifulSoup

url = "https://en.wikipedia.org/wiki/Elon_Musk"  # Replace with the URL of the web page you want to scrape
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")

if not os.path.exists('data'): 
    os.makedirs('data')

if not os.path.exists('data/elon.md'):
    elon = soup.text.split('\nSee also')[0]
    with open('data/elon.md', 'w', encoding='utf-8') as f:
        f.write(elon)
else:
    with open('data/elon.md', 'r') as f:
        elon = f.read()


GraphRag 有一套方便的 CLI 命令可供我们使用。我们将从配置系统开始，然后运行索引操作。使用 GraphRag 进行索引是一个更漫长的过程，成本也高得多，因为 GraphRag 不仅仅是计算嵌入，还会进行多次 LLM 调用来分析文本、提取实体并构建图。不过，这是一次性费用。

In [ ]:
import yaml

if not os.path.exists('data/graphrag'):
    !python -m graphrag.index --init --root data/graphrag

with open('data/graphrag/settings.yaml', 'r') as f:
    settings_yaml = yaml.load(f, Loader=yaml.FullLoader)
settings_yaml['llm']['model'] = "gpt-4o"
settings_yaml['llm']['api_key'] = AZURE_OPENAI_API_KEY if AZURE else OPENAI_API_KEY
settings_yaml['llm']['type'] = 'azure_openai_chat' if AZURE else 'openai_chat'
settings_yaml['embeddings']['llm']['api_key'] = AZURE_OPENAI_API_KEY if AZURE else OPENAI_API_KEY
settings_yaml['embeddings']['llm']['type'] = 'azure_openai_embedding' if AZURE else 'openai_embedding'
settings_yaml['embeddings']['llm']['model'] = TEXT_EMBEDDING_3_LARGE_NAME if AZURE else 'text-embedding-3-large'
if AZURE:
    settings_yaml['llm']['api_version'] = AZURE_OPENAI_API_VERSION
    settings_yaml['llm']['deployment_name'] = GPT4O_DEPLOYMENT_NAME
    settings_yaml['llm']['api_base'] = AZURE_OPENAI_ENDPOINT
    settings_yaml['embeddings']['llm']['api_version'] = AZURE_OPENAI_API_VERSION
    settings_yaml['embeddings']['llm']['deployment_name'] = TEXT_EMBEDDING_3_LARGE_NAME
    settings_yaml['embeddings']['llm']['api_base'] = AZURE_OPENAI_ENDPOINT

with open('data/graphrag/settings.yaml', 'w') as f:
    yaml.dump(settings_yaml, f)

if not os.path.exists('data/graphrag/input'):
    os.makedirs('data/graphrag/input')
    !cp data/elon.md data/graphrag/input/elon.txt
    !python -m graphrag.index --root ./data/graphrag

You should get an output:
🚀 All workflows completed successfully.

To query GraphRag we'll use its CLI again, making sure to configure it with a context length equivalent to what we use in our embeddings search.

In [6]:
import subprocess
import re
DEFAULT_RESPONSE_TYPE = 'Summarize and explain in 1-2 paragraphs with bullet points using at most 300 tokens'
DEFAULT_MAX_CONTEXT_TOKENS = 10000

def remove_data(text):
    return re.sub(r'\[Data:.*?\]', '', text).strip()


def ask_graph(query,method):
    env = os.environ.copy() | {
      'GRAPHRAG_GLOBAL_SEARCH_MAX_TOKENS': str(DEFAULT_MAX_CONTEXT_TOKENS),
    }
    command = [
      'python', '-m', 'graphrag.query',
      '--root', './data/graphrag',
      '--method', method,
      '--response_type', DEFAULT_RESPONSE_TYPE,
      query,
    ]
    output = subprocess.check_output(command, universal_newlines=True, env=env, stderr=subprocess.DEVNULL)
    return remove_data(output.split('Search Response: ')[1])

GrpahRag offers 2 types of search:
1. Global Search for reasoning about holistic questions about the corpus by leveraging the community summaries.
2. Local Search for reasoning about specific entities by fanning-out to their neighbors and associated concepts.

Let's check the local search:

In [8]:
from IPython.display import Markdown
local_query="What and how many companies and subsidieries founded by Elon Musk"
local_result = ask_graph(local_query,'local')

Markdown(local_result)

Elon Musk has founded several companies and subsidiaries across various industries. Here's a summary:

- **SpaceX**: Founded in 2002, SpaceX is a private aerospace manufacturer and space transportation company. Musk serves as the CEO and chief engineer .

- **Tesla, Inc.**: Although not originally founded by Musk, he became an early investor and later the CEO and product architect, significantly shaping its direction .

- **Neuralink**: Co-founded by Musk, this company focuses on developing brain-machine interfaces to enhance human-computer interaction .

- **The Boring Company**: Founded by Musk, it specializes in tunnel construction and innovative transportation solutions .

- **X.com/PayPal**: Musk co-founded X.com, which later became PayPal after merging with Confinity .

- **Zip2**: Co-founded with his brother Kimbal, this was Musk's first venture, later acquired by Compaq .

- **SolarCity**: Co-created by Musk, it was later acquired by Tesla and rebranded as Tesla Energy .

- **xAI**: Founded in 2023, this company focuses on artificial intelligence research .

- **OpenAI**: Co-founded by Musk, this nonprofit organization is dedicated to AI research .

In total, Musk has founded or co-founded at least nine companies and subsidiaries.

In [9]:
global_query="What are the major accomplishments of Elon Musk?"
global_result = ask_graph(global_query,'global')

Markdown(global_result)

Elon Musk has achieved significant accomplishments across various industries, demonstrating his influence and innovation:

- **Space Exploration**: Founder, CEO, and chief engineer of SpaceX, Musk has propelled the company to the forefront of space exploration and satellite deployment, establishing it as a leading spaceflight services provider .

- **Automotive Industry**: As CEO of Tesla, Musk has driven the company to the forefront of electric vehicles and sustainable energy, significantly impacting the automotive industry with innovations in electric cars and energy solutions .

- **Online Payments**: Co-founded X.com, which evolved into PayPal, revolutionizing online transactions and becoming a major player in the online payment industry .

- **Neural Technology**: Co-founded Neuralink, focusing on advancing brain-machine interface technology to enhance the connection between the human brain and computers .

- **Infrastructure**: Founded The Boring Company, specializing in tunnel construction to reduce traffic congestion through innovative underground transportation systems .